In [24]:
import $ivy.`org.apache.kafka:kafka-clients:2.3.0`
import $ivy.`com.google.code.gson:gson:2.8.5`

import $ivy.$                                     

import $ivy.$                                

## Creating Event Class

In [55]:
import java.time.{Instant, ZoneId, ZonedDateTime}
import com.google.gson.Gson

class Event {
    
    var id : Int = -1
    var name : String = _
    var maxPeopleAllowed : Int = _
    var eventType : String = _
    var location : String = _
    var lastUpdate : String = _ 
    
    def this(id : Int, name : String, maxPeopleAllowed : Int, eventType : String, location : String) = {
        this()
        this.id = id
        this.name = name
        this.maxPeopleAllowed = maxPeopleAllowed
        this.eventType = eventType
        this.location = location
        this.lastUpdate = Instant.ofEpochMilli(System.currentTimeMillis).toString
    }
    
    def setUpdates(updates : Map[String,String]) : Unit = {
        
        updates.foreach(m => {
            val k = m._1
            val v = m._2
            
            k match {
                case "id" => println("Not Allowed to Change ID")
                case "name" => this.setName(v)
                case "maxPeopleAllowed" => this.setMaxPeopleAllowed(v.toInt)
            }
            this.update()
        })
    }
    
    private def setName(name : String) : Unit = {
        this.name = name
    }
    
    private def setMaxPeopleAllowed(maxPeopleAllowed : Int) : Unit = {
        this.maxPeopleAllowed = maxPeopleAllowed
    }
    
    private def update() : Unit = {
        this.lastUpdate = Instant.ofEpochMilli(System.currentTimeMillis).toString
    }

}

object Generator {
    private val random = new scala.util.Random
    
    private def randomInt(num : Int) : Int = {
        this.random.nextInt(num)
    }
    
    def getId() : Int = {
        this.randomInt(100)
    }
    
    def getLocation() : String = {
        val randomPos = this.randomInt(6)
        print(randomPos)
        List("San Francisco","Washington","Brazil","London","Barcelona","Madrid","Los Angeles")(randomPos)
    }

}

import java.time.{Instant, ZoneId, ZonedDateTime}

import com.google.gson.Gson


defined class Event
defined object Generator

## Creating Kafka Producer

In [22]:
//Serializer org.apache.kafka.common.serialization.StringSerializer
import org.apache.kafka.clients.producer.{KafkaProducer,ProducerRecord,ProducerConfig,RecordMetadata,Callback}
import java.util.Properties

import org.apache.kafka.clients.producer.{KafkaProducer,ProducerRecord,ProducerConfig,RecordMetadata,Callback}

import java.util.Properties

In [53]:
val jsonParser = new Gson()

jsonParser: Gson = {serializeNulls:false,factories:[Factory[typeHierarchy=com.google.gson.JsonElement,adapter=com.google.gson.internal.bind.TypeAdapters$29@65d60c8], com.google.gson.internal.bind.ObjectTypeAdapter$1@5446cd2, com.google.gson.internal.Excluder@8e8cf73, Factory[type=java.lang.String,adapter=com.google.gson.internal.bind.TypeAdapters$16@3de09fe], Factory[type=java.lang.Integer+int,adapter=com.google.gson.internal.bind.TypeAdapters$7@6ee8726d], Factory[type=java.lang.Boolean+boolean,adapter=com.google.gson.internal.bind.TypeAdapters$3@486ca19e], Factory[type=java.lang.Byte+byte,adapter=com.google.gson.internal.bind.TypeAdapters$5@6317cb7], Factory[type=java.lang.Short+short,adapter=com.google.gson.internal.bind.TypeAdapters$6@75531476], Factory[type=java.lang.Long+long,adapter=com.google.gson.internal.bind.TypeAdapters$11@58056bab], Factory[type=java.lang.Double+double,adapter=com.google.gson.Gson$1@7d35fd54], Factory[type=java.lang.Float+float,adapter=com.google.gson.Gson$

In [43]:
val producerProperties = new Properties()
producerProperties.put(ProducerConfig.BOOTSTRAP_SERVERS_CONFIG,"localhost:9092")
producerProperties.put(ProducerConfig.KEY_SERIALIZER_CLASS_CONFIG,"org.apache.kafka.common.serialization.StringSerializer")
producerProperties.put(ProducerConfig.VALUE_SERIALIZER_CLASS_CONFIG,"org.apache.kafka.common.serialization.StringSerializer")
producerProperties.put(ProducerConfig.ACKS_CONFIG,"1") // Only Leader of Partition

val eventProducer = new KafkaProducer[String,String](producerProperties,)

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


producerProperties: Properties = {bootstrap.servers=localhost:9092, acks=1, value.serializer=org.apache.kafka.common.serialization.StringSerializer, key.serializer=org.apache.kafka.common.serialization.StringSerializer}
res42_1: Object = null
res42_2: Object = null
res42_3: Object = null
res42_4: Object = null
eventProducer: KafkaProducer[String, String] = org.apache.kafka.clients.producer.KafkaProducer@15f013bd

In [56]:
val eventsArray = new scala.collection.mutable.ArrayBuffer[Event]()
eventsArray.append(new Event(Generator.getId,"AWS Summit",600,"Technology",Generator.getLocation))
eventsArray.append(new Event(Generator.getId,"Azure Conference",850,"Technology",Generator.getLocation))
eventsArray.append(new Event(Generator.getId,"JavaScript Conference",200,"Technology",Generator.getLocation))
eventsArray.append(new Event(Generator.getId,"Anime Friends",410,"Animes",Generator.getLocation))
eventsArray.append(new Event(Generator.getId,"Comic Con",410,"Enterneinement",Generator.getLocation))

02253

eventsArray: collection.mutable.ArrayBuffer[Event] = ArrayBuffer(
  ammonite.$sess.cmd54$Helper$Event@22d31b28,
  ammonite.$sess.cmd54$Helper$Event@7f48573c,
  ammonite.$sess.cmd54$Helper$Event@34ec7cf5,
  ammonite.$sess.cmd54$Helper$Event@6a36d2e6,
  ammonite.$sess.cmd54$Helper$Event@58b466d5
)

In [61]:
eventsArray(0).setUpdates(Map("name" -> "AWS Summit 2019"))

In [62]:
for (i <- 0 until eventsArray.length) {
    var event = eventsArray(i)
    var json = jsonParser.toJson(event)
    var message = new ProducerRecord[String,String]("animes_topic",json)
    eventProducer.send(message,new Callback {
        override def onCompletion(recordMetadata : RecordMetadata, e : Exception) : Unit = {
            if( e == null) {
                println(e)
            } else {
                println(s"Event Sent! : Partition : ${recordMetadata.offset()}, Enqeued Time : ${recordMetadata.timestamp}")
            }
        }
    })
}

null